In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import pypsa

In [ ]:
RUNS = { "noghc": "decr_14_3h_ws",
        "annual": "decr_16_3h_ws",
        "monthly": "decr_15_3h_ws",
        "hourly": "decr_13_3h_ws",
       }

PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
#OUTPUT = PREFIX + "results/" + RUN + "/graphics/comp_benefits/"

In [ ]:
def get_data(stats, h2export, opts):
    """Filter stats according to h2export value and opts value
    """
    cost = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"cost"]
    mg_AC_exclu_H2 = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"mg_AC_exclu_H2 El_all"]
    mg_H2_False_False_exportonly = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"mg_H2_False_False_exportonly"]
    exp_AC_exclu_H2 = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"exp_AC_exclu_H2 El_all"]
    exp_H2_False_False_exportonly = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"exp_H2_False_False_exportonly"]

    return cost, mg_AC_exclu_H2, mg_H2_False_False_exportonly, exp_AC_exclu_H2, exp_H2_False_False_exportonly

In [ ]:
def prepare_data(ghrules, h2export, opts):
    df = pd.DataFrame(index=ghrules, columns=["cost", "mg_AC_exclu_H2", "mg_H2_False_False_exportonly"])

    for rule in ghrules:
        INPUT = PREFIX + "results/" + RUNS[rule] + "/integrated_comp/stats.csv"
        stats = pd.read_csv(INPUT, index_col=0)
        cost, mg_AC_exclu_H2, mg_H2_False_False_exportonly, exp_AC_exclu_H2, exp_H2_False_False_exportonly = get_data(stats, h2export, opts)    
        df.loc[rule, "cost"] = cost.values[0]
        df.loc[rule, "mg_AC_exclu_H2"] = mg_AC_exclu_H2.values[0]
        df.loc[rule, "mg_H2_False_False_exportonly"] = mg_H2_False_False_exportonly.values[0]    
        df.loc[rule, "exp_AC_exclu_H2"] = exp_AC_exclu_H2.values[0]
        df.loc[rule, "exp_H2_False_False_exportonly"] = exp_H2_False_False_exportonly.values[0]

    # Add relative values in % based on noghc row
    df["cost_rel"] = df["cost"] / df.loc["noghc", "cost"] * 100
    df["mg_AC_exclu_H2_rel"] = df["mg_AC_exclu_H2"] / df.loc["noghc", "mg_AC_exclu_H2"] * 100
    df["mg_H2_False_False_exportonly_rel"] = df["mg_H2_False_False_exportonly"] / df.loc["noghc", "mg_H2_False_False_exportonly"] * 100
    return df  

### Relative, subplot

In [ ]:
h2exports = [200, 1]
opts = ["Co2L2.0", "Co2L0.00"]
ghrules = ["noghc","annual", "monthly", "hourly"]

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 6), sharex=True, sharey=True)

n=1
for h2export in h2exports:
    for o in opts:
        ax = plt.subplot(2, 2, n)
        n += 1

        df = prepare_data(ghrules, h2export, o)

        ax.plot(df.index, df["cost_rel"], label="Total costs", color="black")
        ax.plot(df.index, df["mg_H2_False_False_exportonly_rel"], label="H2 costs paid by exporters", color="orange")
        ax.plot(df.index, df["mg_AC_exclu_H2_rel"], label="Local electricity price", color="green")

        ax.set_title(f"{h2export} TWh export, {o} CO2")
        ax.grid(axis='y')
        ax.set_ylim(0, 135)

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc = (0.4,-0.01))
fig.text(0.04, 0.5, 'Change in %', va='center', rotation='vertical')

plt.show()

### Relative, multiple in one

In [ ]:
h2exports = [1, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200]
#opts = ["Co2L0.50"]
opts = ["Co2L0.40"] #["Co2L2.0", "Co2L0.90", "Co2L0.80", "Co2L0.70", "Co2L0.60", "Co2L0.50", "Co2L0.40", "Co2L0.30", "Co2L0.20", "Co2L0.10", "Co2L0.00"]
ghrules = ["noghc","annual", "monthly", "hourly"]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

n=1
for h2export in h2exports:
    for o in opts:
        
        n += 1

        df = prepare_data(ghrules, h2export, o)

        ax.plot(df.index, df["mg_H2_False_False_exportonly_rel"], label="H2 costs paid by exporters", color="orange", linewidth=1,  alpha=1-(h2export/200)*0.7)
        ax.plot(df.index, df["mg_AC_exclu_H2_rel"], label="Local electricity price", color="green", linewidth=1, alpha=1-(h2export/200)*0.7)
        ax.plot(df.index, df["cost_rel"], label="Total costs", color="black", linewidth=1, alpha=1)

ax.set_title(f"{min(h2exports)} - {max(h2exports)} TWh export, {opts[:1]} - {opts[-1:]} CO2")
ax.grid(axis='y')
ax.set_ylim(0, 135)

handles, labels = ax.get_legend_handles_labels()
#fig.legend(handles, labels, loc = (0.4,-0.01))
ax.set_ylabel("Change in % compared to no H2 rule")

plt.show()

### Relative, single

In [ ]:
df = prepare_data(ghrules, 200, "Co2L2.0")
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(df.index, df["cost_rel"], label="Total costs", color="black")
ax.plot(df.index, df["mg_H2_False_False_exportonly_rel"], label="H2 costs paid by exporters", color="orange")
ax.plot(df.index, df["mg_AC_exclu_H2_rel"], label="Local electricity price", color="green")

ax.set_xlabel("H2 rule")
ax.set_ylabel("Change in % compared to no H2 rule")
ax.set_title(f"200 TWh export, 0% CO2 reduction")
ax.set_ylim(0, 135)
ax.grid(axis='y')

ax.legend(loc="lower left")

### Absolute prices

In [ ]:
df = prepare_data(ghrules, 200, "Co2L2.0")
fig, ax = plt.subplots(figsize=(6, 4))

#ax.plot(df.index, df["cost"]/1e9, label="Total cost")
ax.plot(df.index, df["mg_AC_exclu_H2"], label="Local electricity price", color="green")
ax.plot(df.index, df["mg_H2_False_False_exportonly"], label="H2 costs paid by exporters", color="orange")


# Add secondary axis for total cost
ax2 = ax.twinx()
ax2.plot(df.index, df["cost"]/1e9, label="Total cost", color="black")
ax2.set_ylabel("Total cost in B€")
ax2.set_ylim(5, 30)

ax.set_xlabel("H2 rule")
ax.set_ylabel("Marginal price of electricity/h2 in €/MWh")
ax.set_ylim(0, 100)

ax.legend(loc="lower left")
ax2.legend(loc="lower right")

ax.grid(axis='y')

### Absolute expenses

In [ ]:
h2export = 200
opts = "Co2L2.0"

In [ ]:
df = prepare_data(ghrules, h2export, opts)
fig, ax = plt.subplots(figsize=(6, 4))

width = 0.4
ax.bar(df.index, df["exp_H2_False_False_exportonly"]/1e3, width=width, label="H2 expenses (exporter)", color="orange", alpha=0.7)
ax.bar(df.index, df["exp_AC_exclu_H2"]/1e3, bottom=df["exp_H2_False_False_exportonly"]/1e3, width=width, label="Electricity expenses (local)", color="green", alpha=0.7)
ax.plot(df.index, df["cost"]/1e9, label="Total cost", color="black")

ax.set_ylabel("Expense in B€")
#ax.set_ylim(0, 27)

ax.set_xlabel("H2 rule")
ax.set_title(f"{h2export} TWh export, {opts} CO2")

ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

ax.grid(axis='y', alpha=0.5)   

### All average nodal prices

In [ ]:
def get_mean_prices(stats, h2export, opts):
    """Filter stats according to h2export value and opts value
    """
    stats = stats[(stats.index == h2export) & (stats.opts == opts)]
    stats = stats.iloc[:, stats.columns.str.startswith("mean_mg")]

    return stats

In [ ]:
def prepare_mean_prices(ghrules, h2export, opts):
    df = pd.DataFrame() #index=ghrules)

    for rule in ghrules:
        INPUT = PREFIX + "results/" + RUNS[rule] + "/integrated_comp/stats.csv"
        stats = pd.read_csv(INPUT, index_col=0)
        prices = get_mean_prices(stats, h2export, opts)    
        prices.index = [rule]
        df = pd.concat([df, prices])

    return df  

In [ ]:
h2export = 200
opts = "Co2L2.0"
increase = 0.04 # Percentage of increase above which it is considered as increase. Set "False" to allow all marginal prices

In [ ]:
df = prepare_mean_prices(ghrules, h2export, opts)

if increase != False:
    # Filter df for columns which have higher values in any row (apart from "noghc") than in "noghc"
    df = df.loc[:, (df.loc[["hourly", "monthly", "annual"]] > (df.loc["noghc"]*(1+increase/100))).any()]

fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(df.index, df, label=df.columns.str.replace("mean_mg_", ''))

ax.set_xlabel("H2 rule")
ax.set_ylabel("Average marginal price (non-weighted)")
ax.set_title(f"{h2export} TWh export, {opts} CO2")

ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

ax.grid(axis='y')

In [ ]:
df

In [ ]:
EXPORT = 200
CO2L = 2.0
CLUSTERS = 4
H = 3
LL = "3.0"
DISCOUNT_RATE = 0.13

RUN = "decr_14_3h_ws"
CATEGORY = "postnetworks"
SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_Co2L{CO2L}_{H}H_2030_{DISCOUNT_RATE}_DF_{EXPORT}export"

#OVERRIDES = PATH + "pypsa-eur-sec/data/override_component_attrs"

PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
OUTPUT = PREFIX + "results"
INPUT = PREFIX + f"workflow/subworkflows/pypsa-earth-sec/results/{RUN}/{CATEGORY}/{SCENARIO}.nc"
OUTPUT_SCENARIO = f"{OUTPUT}/{RUN}/graphics/demand/{SCENARIO}"
OUTPUT_RAW = f"{OUTPUT}/{RUN}/graphics/demand/"

In [ ]:
n = pypsa.Network(INPUT)

### System cost by expense

In [ ]:
# TODO simplify this. df.loc["noghc"] is the same as -(n.statistics.revenue(comps=["Load"]) /1e9).droplevel(0)

In [ ]:
n.objective / 1e9

In [ ]:
def get_expense_carrier(n, excl_em=True):

    # Multiply demands and prices
    loads = n.loads_t.p[n.loads.index]
    loads.columns = n.loads.bus # Rename columns to bus names to comply with columns of "prices"
    prices = n.buses_t.marginal_price[n.loads.bus]
    expenses = loads * prices * n.snapshot_weightings.generators[0]

    nodal_expense = expenses.sum() #.groupby(n.loads.bus).sum() / 1e9
    nodal_expense.name = "expense"

    # concat loads_carrier and nodal_expense based on index
    loads_carrier = n.loads.carrier
    loads_carrier.index = n.loads.bus 
    nodal_expense_m = pd.concat([nodal_expense, loads_carrier], axis=1)

    expense_c = nodal_expense_m.groupby("carrier").sum() / 1e9

    if excl_em == True:
        expense_c = expense_c[~expense_c.index.str.contains("emission")]
    else:
        pass

    return expense_c

In [ ]:
def get_expense_carriers(ghrules, h2export, opts, excl_em):
    """Get expense per carrier
    """

    df = pd.DataFrame()

    for rule in ghrules:
        INPUT = PREFIX + f"workflow/subworkflows/pypsa-earth-sec/results/{RUNS[rule]}/{CATEGORY}/{SCENARIO}.nc"

        n = pypsa.Network(INPUT)

        expense_c = get_expense_carrier(n, excl_em=excl_em)
        expense_c = expense_c.T
        expense_c.index = [rule]
        expense_c.columns.name = None

        df = pd.concat([df, expense_c])

        
    return df

In [ ]:
def prep_expense_carriers(df):
   # Combine the columns containing "AC" or "electricity" to one col and delete the others
   combine_cols = df.columns[df.columns.str.contains("electricity")]
   ev = "land transport EV"

   if ev in df.columns:
      df["Local electricity"] =  df[combine_cols].sum(axis=1) + df["AC"] + df["land transport EV"]
      df = df.drop(columns=["AC", "land transport EV"])
   else:
      df["Local electricity"] =  df[combine_cols].sum(axis=1) + df["AC"]
      df = df.drop(columns=["AC"])
   
   df = df.drop(columns=combine_cols)

   move = ["H2", "Local electricity"]
   keep = df.columns[~df.columns.isin(move)]
   df = df[move + list(keep)]

   return df

In [ ]:
df = get_expense_carriers(ghrules, h2export, opts, excl_em=True)

In [ ]:
df.loc["noghc"] # same as -(n.statistics.revenue(comps=["Load"]) /1e9).droplevel(0)

In [ ]:
df_plot = prep_expense_carriers(df)

In [ ]:
df.sum(axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

df = prepare_data(ghrules, 200, "Co2L2.0")
plt.plot(df.index, df["cost"]/1e9, label="Total cost", color="black")

df_plot.plot.bar(ax=ax, stacked=True, alpha=0.7)
plt.xlabel("H2 rule")
plt.ylabel("Expense in B€")
plt.title(f"{h2export} TWh export, {CO2L} CO2")
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(axis='y', alpha=0.5) 

### System costs by technology

In [ ]:
# Theory: total system cost = n.objective
(n.statistics.capex().sum() - n.statistics.installed_capex().sum()) - (n.statistics.capex() - n.statistics.installed_capex()).sum()

In [ ]:
def get_costs(threshold, ghrules, h2export, opts):
    """Get costs
    """

    costs = pd.DataFrame()

    for rule in ghrules:
        INPUT = PREFIX + f"workflow/subworkflows/pypsa-earth-sec/results/{RUNS[rule]}/{CATEGORY}/{SCENARIO}.nc"

        n = pypsa.Network(INPUT)

        capex = n.statistics.capex() - n.statistics.installed_capex()
        threshold_sum_capex = capex[capex < threshold].sum() / 1e9
        capex = capex[capex > threshold] / 1e9 # in B€
        capex = pd.DataFrame(capex.droplevel(0)).T
        capex = capex.add_suffix(" (CAPEX)")
        capex[f"(CAPEX < {threshold/1e6} M€ thres.)"] = threshold_sum_capex

        opex = n.statistics.opex()
        threshold_sum_opex = opex[opex < threshold].sum() / 1e9
        opex = opex[opex > threshold] / 1e9
        opex = pd.DataFrame(opex.droplevel(0)).T
        opex = opex.add_suffix(" (OPEX)")
        opex[f"(OPEX < {threshold/1e6} M€ thres.)"] = threshold_sum_opex
        opex =opex.groupby(opex.columns, axis=1).sum()

        caop = pd.concat([capex, opex], axis=1)

        caop.index = [rule]

        costs = pd.concat([costs, caop], axis=0) #, ignore_index=True)

    
    return costs

In [ ]:
threshold = 10e6 # 10 M€

In [ ]:
costs = get_costs(threshold, ghrules, h2export, opts)

In [ ]:
df = prepare_data(ghrules, 200, "Co2L2.0")

fig, ax = plt.subplots(figsize=(6, 4))

costs.plot.bar(ax=ax, stacked=True, alpha=0.7)
plt.plot(df.index, df["cost"]/1e9, label="Total cost", color="black")

plt.xlabel("H2 rule")
plt.ylabel("Costs in B€")
plt.title(f"{h2export} TWh export, {CO2L} CO2")
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(axis='y', alpha=0.5) 

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

# Divide all values by first row
costs_rel = costs / costs.iloc[0] *100

costs_rel.plot.line(ax=ax, alpha=0.7)
#plt.plot(df.index, df["cost"]/1e9, label="Total cost", color="black")

plt.xlabel("H2 rule")
plt.ylabel("Relative change in %")
plt.title(f"{h2export} TWh export, {CO2L} CO2")
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(axis='y', alpha=0.5) 

### n.statistics() tests

CAPEX and OPEX: works

In [ ]:
n.objective / 1e9 - (n.statistics.capex().sum() + n.statistics.opex().sum() - n.statistics.installed_capex().sum() )/ 1e9

In [ ]:
n.statistics.opex().sort_values() / 1e9

Revenue

In [ ]:
# Expenses
n.statistics.revenue()[n.statistics.revenue() <0].sum() / 1e9

In [ ]:
# Incomes
n.statistics.revenue()[n.statistics.revenue() >0].sum() / 1e9

In [ ]:
n.statistics.revenue()[n.statistics.revenue() >0].sort_values() / 1e9

In [ ]:
# n.statistics.revenue(comps=["Load"]).droplevel(0).div(1e9).plot.bar(
#     title="Revenue from load",
# )

In [ ]:
n.statistics.revenue(comps=["Load"]).sort_values() / 1e9

In [ ]:
n.statistics.revenue(comps=["Load"])[n.statistics.revenue(comps=["Load"]).droplevel(0).index.str.contains("oil")].sort_values().sum() / 1e9

In [ ]:
n.statistics.revenue(comps=["Store"]).sort_values()  / 1e9

In [ ]:
n.statistics.revenue(comps=["Store"])[n.statistics.revenue(comps=["Store"]) <0].sum() / 1e9

In [ ]:
n.statistics.revenue(comps=["Store"])[n.statistics.revenue(comps=["Store"]) >0].sum() / 1e9

Misc